<a href="https://colab.research.google.com/github/JinFree/PDE-for-CFD-in-Python/blob/master/Parabolic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import os
import shutil
import numpy as np
import numba